# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import psycopg2
import pandas as pd
from sql_queries import *
from typing import Iterable
from pathlib import Path
from psycopg2.extras import execute_values

In [4]:
# conn = psycopg2.connect(host='localhost', dbname='sparkifydb', user='student', password='student')
conn = psycopg2.connect(host='localhost', dbname='sparkifydb')
cur = conn.cursor()

In [6]:
def get_files(filepath: Path) -> Iterable[Path]:
    root = Path(filepath).resolve()
    all_paths = list(root.rglob('*.json'))

    return all_paths

## Process `song_data`
Perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files('data/song_data')

In [10]:
filepath = song_files[0]

In [12]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


### #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [14]:
song_df = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_df.head()

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [16]:
song_tuples = list(tuple(x) for x in song_df.to_numpy())

In [18]:
execute_values(cur, song_table_insert, song_tuples)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

### #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [20]:
artist_df = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_df.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [22]:
artist_tuples = list(tuple(x) for x in artist_df.to_numpy())

In [24]:
execute_values(cur, artist_table_insert, artist_tuples)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [26]:
log_files = get_files('data/log_data')

In [28]:
filepath = log_files[0] 

In [30]:
df = pd.read_json(filepath, lines=True)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access datetimelike properties easliy.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [32]:
df = df[df.page == 'NextSong']
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [34]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [36]:
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_data = [(str(x), x.hour, x.day, x.week, x.month, x.year, x.dayofweek) for x in t.to_list()]
time_data[0]

('2018-11-11 02:33:56.796000', 2, 11, 45, 11, 2018, 6)

In [38]:
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head(2)

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796000,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796000,2,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [40]:
time_tuples = [tuple(x) for x in time_df.to_numpy()]

In [42]:
execute_values(cur, time_table_insert, time_tuples)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [44]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].copy()
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [46]:
user_tuples = [tuple(x) for x in user_df.to_numpy()]

In [48]:
execute_values(cur, user_table_insert, user_tuples)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [50]:
for _, row in df.iterrows():

    # get song_id and artist_id from song and artist tables
    cur.execute(song_select, (row.song, row.artist, float(row.length)))
    results = cur.fetchone()
    
    song_id, artist_id = results if results else (None, None)

    # insert songplay record
    songplay_data = (
        str(pd.to_datetime(row.ts, unit='ms')),
        row.userId,
        row.level,
        song_id,
        artist_id,
        row.sessionId,
        row.location,
        row.userAgent
    )
    execute_values(cur, songplay_table_insert, [songplay_data])
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

### Close Connection to Sparkify Database

In [52]:
cur.close()
conn.close()